In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2 
from sklearn import preprocessing
import os
from torchvision import datasets, models, transforms

#if linux
import matplotlib
matplotlib.use('TkAgg')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 
print(torch.version.cuda)
print("GPU:",torch.cuda.is_available())



class SimpleNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size,layers=[1000,500,200],drop_out_prob=0.2):
        super(SimpleNeuralNetwork, self).__init__()
        self.fc=[nn.Linear(input_size, layers[0])]
        self.fc.append(nn.Sigmoid())
        self.fc.append(nn.Dropout(p=drop_out_prob))
        for i in range(1,len(layers)): #create layers 
                self.fc.append(nn.Linear(layers[i-1], layers[i]))
                self.fc.append(nn.Sigmoid())
                self.fc.append(nn.Dropout(p=drop_out_prob))
        self.fc.append(nn.Linear(layers[-1], output_size))
        self.fc_layers = nn.Sequential(*self.fc)
    def forward(self, x):
        x=self.fc_layers(x)
        return x
    
class SimpleConv2DNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size,layers=[1000,500,200],drop_out_prob=0.2):
        super(SimpleConv2DNeuralNetwork, self).__init__()
        input_channels=1 #greyscale
        self.conv_layer = nn.Conv2d(in_channels=input_channels, out_channels=8, kernel_size=3)
        # Add 2D convolutional layer
        self.conv_layer = nn.Conv2d(in_channels=input_channels, out_channels=8, kernel_size=3)
        
        # Calculate the size of the tensor after convolution and pooling
        conv_output_size = self._get_conv_output_size(input_channels, input_size[0], input_size[1])  # Adjust the last two dimensions
        # Add hidden layers
        self.fc=[nn.Linear(int(conv_output_size), layers[0])]
        self.fc.append(nn.Sigmoid())
        self.fc.append(nn.Dropout(p=drop_out_prob))
        for i in range(1,len(layers)): #create layers 
                self.fc.append(nn.Linear(layers[i-1], layers[i]))
                self.fc.append(nn.Sigmoid())
                self.fc.append(nn.Dropout(p=drop_out_prob))
        self.fc.append(nn.Linear(layers[-1], output_size))
        self.fc_layers = nn.Sequential(*self.fc)
    def forward(self, x):
        x=self.conv_layer(x)
        x = torch.relu(x)
        # Reshape the tensor to match the input size of the first fully connected layer
        x = x.view(x.size(0), -1)
        x=self.fc_layers(x)
        return x
    def _get_conv_output_size(self, input_channels, height, width):
        dummy_input = torch.zeros(1, input_channels, height, width)
        dummy_output = self.conv_layer(dummy_input)
        return dummy_output.view(-1).size(0)

n=SimpleConv2DNeuralNetwork([149,192],266)
x=n.forward(torch.tensor(np.random.normal(0,1,(50,1,149,192))).float())
x.shape

11.7
GPU: True


torch.Size([50, 266])

In [2]:

name="pickle_linear.pkl"
file="/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/TacTip reader/dots/" #"C:/Users/dexte/github/RoboSkin/Code/Models/TacTip reader/dots/"

#load dataset
y=np.load(file+"augmentedNewTacTip.npy")
X_train=np.load(file+"augmentedNewTacTipimages.npy")

DATASET_SIZE=300 #X_train.shape[0]
print("Training dataset size",len(X_train))
print("Labels dataset size",len(y))
#shape correctly
SIZE=0.3
y=y[0:DATASET_SIZE].reshape((y[0:DATASET_SIZE].shape[0],y[0:DATASET_SIZE].shape[1]*2))
y=y/(255)*SIZE
h=X_train.shape[2]*SIZE
w=X_train.shape[1]*SIZE
#this is the part to apply preprocessing
X_train_grayscale = np.zeros((DATASET_SIZE,int(w),int(h))) # ((X_train.shape[0],int(h*w))) #((X_train.shape[0],144,192))
for i in range(DATASET_SIZE):
    im=X_train[i].copy()
    im=cv2.resize(im,(int(h),int(w)),interpolation=cv2.INTER_AREA)
    im = np.uint8(im)
    current = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #
    frame = cv2.adaptiveThreshold(
            current, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 5, 1
        )
    kernel = np.ones((2, 2), np.uint8)
    frame = cv2.erode(frame, kernel, iterations=1)
    X_train_grayscale[i]=frame/255 #remove flatten for conv


plt.imshow(cv2.resize(X_train_grayscale[0].reshape(144,192),(int(h),int(w)),interpolation=cv2.INTER_AREA))
example=(y[0].reshape((y[-1].shape[0]//2,2))*255)
plt.scatter(example[:,0],example[:,1])
#plt.show()

Training dataset size 990
Labels dataset size 990


In [3]:
X, data_test, Y, labels_test = train_test_split(X_train_grayscale, y, test_size=0.20, random_state=42)
X.shape,Y.shape
#X=X.reshape(792,1,144,192)
#data_test=data_test

((240, 144, 192), (240, 266))

In [8]:
torch.cuda.empty_cache() 

# Define the size of the input (n) and output (m) layers
n_inputs = X.shape[1]
m_outputs = Y.shape[1]
print(X.shape)
# Convert data to PyTorch tensors
X_tensor = torch.tensor(X).view(X.shape[0],1,X.shape[1],X.shape[2])#.to(device)#.to(torch.float32)
y_tensor = torch.tensor(Y)#.to(device)#.to(torch.float32)

# Define the neural network model


# Create the neural network
model = SimpleConv2DNeuralNetwork([149,192], m_outputs,layers=[1000],drop_out_prob=0.1)#.to(device)
#model=Network().to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(model,num_epochs,output=True):
    loss_ar=[]
    for epoch in range(num_epochs):
        # Forward pass
        y_pred = model(X_tensor.float())

        # Compute the loss
        loss = criterion(y_pred, y_tensor)

        # Zero gradients, backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_ar.append(loss.item())
        #predict
        # Print the current loss to monitor training progress
        if epoch%1000==0 and output:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    return np.array(loss_ar)
plt.title("Loss while training marker prediction model")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
loss=train(model,150000)
plt.plot(loss,label="Loss")
#plt.plot(b,label="Accuracy")
plt.legend(loc="upper right")
plt.show()

(240, 144, 192)


: 

: 

In [15]:
loss=train(model,20000)

Epoch [1/20000], Loss: 0.0007
Epoch [1001/20000], Loss: 0.0007
Epoch [2001/20000], Loss: 0.0007
Epoch [3001/20000], Loss: 0.0007
Epoch [4001/20000], Loss: 0.0007
Epoch [5001/20000], Loss: 0.0007
Epoch [6001/20000], Loss: 0.0007
Epoch [7001/20000], Loss: 0.0007
Epoch [8001/20000], Loss: 0.0007
Epoch [9001/20000], Loss: 0.0007
Epoch [10001/20000], Loss: 0.0007
Epoch [11001/20000], Loss: 0.0007
Epoch [12001/20000], Loss: 0.0007
Epoch [13001/20000], Loss: 0.0007
Epoch [14001/20000], Loss: 0.0007
Epoch [15001/20000], Loss: 0.0007
Epoch [16001/20000], Loss: 0.0007
Epoch [17001/20000], Loss: 0.0007
Epoch [18001/20000], Loss: 0.0007
Epoch [19001/20000], Loss: 0.0007


In [12]:
np.save("DataLossA",loss)

In [14]:
torch.save(model.state_dict(), "C:/Users/dexte/github/RoboSkin/Code/Models/TacTip reader/pytorchModelCross.pth")

In [26]:
X, data_test, Y, labels_test = train_test_split(X_train_grayscale, y, test_size=0.20, random_state=42)
# Define the size of the input (n) and output (m) layers
n_inputs = X.shape[1]
m_outputs = Y.shape[1]
model = SimpleNeuralNetwork(n_inputs, m_outputs,drop_out_prob=0.1)#.to(device)

model.load_state_dict(torch.load("/its/home/drs25/Documents/GitHub/RoboSkin/Code/Models/TacTip reader/pytorchModel.pth"))
model.eval()


/its/home/drs25/miniconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


SimpleNeuralNetwork(
  (fc_layers): Sequential(
    (0): Linear(in_features=27648, out_features=1000, bias=True)
    (1): Sigmoid()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=1000, out_features=500, bias=True)
    (4): Sigmoid()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=500, out_features=200, bias=True)
    (7): Sigmoid()
    (8): Dropout(p=0.1, inplace=False)
    (9): Linear(in_features=200, out_features=266, bias=True)
  )
)

In [27]:
with torch.no_grad():
     model.eval()
     predictions = model(torch.tensor(data_test, dtype=torch.float32)).detach().numpy()
import random
N=random.randint(0,len(predictions)-101)
for i in range(100):
     example=predictions[N+i]
     example=(example.reshape((example.shape[0]//2,2))*255)/SIZE
     image=data_test[N+i].reshape((int(w),int(h)))
     image=cv2.resize(image,(X_train.shape[2],X_train.shape[1]),interpolation=cv2.INTER_AREA)
     label=(labels_test[N+i].reshape((labels_test[N+i].shape[0]//2,2))*255)/SIZE
     plt.cla()
     plt.imshow(image)
     plt.scatter(example[:,0],example[:,1],c="b")
     plt.scatter(label[:,0],label[:,1],c="r")
     plt.pause(0.5)

Exception in Tkinter callback
Traceback (most recent call last):
  File "/its/home/drs25/miniconda3/lib/python3.10/tkinter/__init__.py", line 1919, in __call__
    if self.subst:
KeyboardInterrupt
